# LangChain Cohere Agent using Tools

## Install Libraries

In [ ]:
# ! pip install streamlit
# ! pip install qdrant-client
# ! pip install langchain
# ! pip install langchain-community
# ! pip install langchain-qdrant
# ! pip install langchain-cohere
# ! pip install tiktoken
# ! pip install cohere
# ! pip install bs4
# ! pip install markdown
# ! pip install wikipedia
# ! pip install pyowm
# ! pip install yfinance

## Imports for Tool Use and Keys
### API Keys
* https://cohere.com/
* https://www.alphavantage.co/
* https://qdrant.tech/
* https://openweathermap.org/
* https://tavily.com/
* https://polygon.io/


In [1]:
from IPython.display import HTML
import markdown
import warnings
import streamlit as st
import os
from langchain_cohere.llms import Cohere
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain.agents import AgentExecutor
from cohere_tools import get_internet_search_tool, get_wikipedia_tool, get_openweathermap_tool
from yfinance_tools import get_business_summary_tool, get_ticker_history_tool, get_ticker_info_tool
from alpha_vantage_tools import (
    get_search_symbol_tool,
    get_exchange_rate_tool,
    # get_time_series_daily_tool,
    # get_time_series_weekly_tool,
    # get_market_news_sentimen_tool,
    # get_top_gainers_losers_tool,
)

warnings.filterwarnings("ignore")

ALPHAVANTAGE_API_KEY = st.secrets["ALPHAVANTAGE_API_KEY"]
os.environ["ALPHAVANTAGE_API_KEY"] = st.secrets["ALPHAVANTAGE_API_KEY"]

COHERE_API_KEY = st.secrets["COHERE_API_KEY"]
os.environ["COHERE_API_KEY"] = st.secrets["COHERE_API_KEY"]

QDRANT_HOST = st.secrets["QDRANT_HOST"]
QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]

OPEN_WEATHER_API_KEY = st.secrets["OPEN_WEATHER_API_KEY"]
os.environ["OPENWEATHERMAP_API_KEY"] = OPEN_WEATHER_API_KEY

POLYGON_API_KEY = st.secrets["POLYGON_API_KEY"]
os.environ["POLYGON_API_KEY"] = st.secrets["POLYGON_API_KEY"]

TAVILY_API_KEY = st.secrets["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = st.secrets["TAVILY_API_KEY"]

## Invoke ChatCohere

In [8]:
chat = ChatCohere()
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages))

content="Who's there?" additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8e78cb0d-af71-425c-aec0-73f4e2ac175f', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8e78cb0d-af71-425c-aec0-73f4e2ac175f', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} id='run-5561524f-54ce-4c9d-bc1e-c55958d9d77a-0'


## Invoke ChatCohere using ChatPromptTemplate

In [9]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat
chain.invoke({"topic": "bears"})

AIMessage(content='Why don’t bears wear socks?\n\nThey have bear feet!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'c9c7e2b2-8d70-4a99-814f-c36f593e3de5', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'c9c7e2b2-8d70-4a99-814f-c36f593e3de5', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, id='run-87c97268-c707-438f-8c86-ccc31919c308-0')

## Invoke Cohere

In [12]:
llm = Cohere()
display(HTML(markdown.markdown(llm.invoke("Summarize the theory of Kahnemann and Tversky"))))

## Invoke Cohere with Tools

In [2]:
def invoke_cohere_with_tools(input, tools, prompt_template="{input}",no_markdown=False):
    llm = ChatCohere()
    prompt = ChatPromptTemplate.from_template(prompt_template)
    agent = create_cohere_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    response = agent_executor.invoke(input)
    if no_markdown:
        print(response['output'])
    else:
        display(HTML(markdown.markdown(response['output'])))



## Tavily Internet Search Tool

In [8]:
prompt_template = "In which year had {stock_name} its IPO? Tell me the year and the date."
input =  {"stock_name": "Amazon"}
tools = [get_internet_search_tool()]
invoke_cohere_with_tools(input, tools, prompt_template)



> Entering new AgentExecutor chain...

I will search for 'Amazon IPO date' and then write an answer.
{'tool_name': 'internet_search', 'parameters': {'query': 'Amazon IPO date'}}
[{'url': 'https://www.cnbc.com/2018/09/05/if-you-invested-1000-in-amazon-in-1997-heres-what-youd-have-now.html', 'content': "From 8/28/2017 to 8/31/2018. As impressive as that is, original investors in Amazon fare even better. If you had invested $1,000 during Amazon's IPO in May 1997, your investment would be worth ..."}, {'url': 'https://www.fool.com/investing/2022/09/22/invested-10000-in-amazon-ipo-in-1997-how-much-now/', 'content': 'Amazon went public on May 15, 1997, at $18 per share. A $10,000 investment in Amazon at its IPO would be worth over $16 million today, thanks to its online marketplace, Prime membership, and cloud business.'}, {'url': 'https://finance.yahoo.com/news/heres-much-investing-1-000-151649171.html', 'content': "On May 15, 1997, Amazon.com, Inc. (NASDAQ: AMZN) held its initial public 

## Wikipedia Tool

In [9]:
input =  {"input": "Explain the theory of Kahnemann and Tversky"}
tools = [get_wikipedia_tool()]
invoke_cohere_with_tools(input, tools)



> Entering new AgentExecutor chain...

I will search for 'Kahnemann and Tversky theory' and write an answer based on the information I find.
{'tool_name': 'wikipedia-tool', 'parameters': {'query': 'Kahnemann and Tversky theory'}}
Page: Daniel Kahneman
Summary: Daniel Kahneman (; Hebrew: דניאל כהנמן; March 5, 1934 – March 27, 2024) was an Israeli-American cognitive scientist best-known for his work on the psychology of judgment and decision-making. He is also known for his work in behavioral economics, for which he was awarded the 2002 Nobel Memorial Prize in Economic Sciences together with Vernon L. Smith. Kahneman's published empirical findings challenge the assumption of human rationality prevailing in modern economic theory. Kahneman became known as the "grandfather of behavioral economics."
With Amos Tversky and others, Kahneman established a cognitive basis for common human errors that arise from heuristics and biases, and developed prospect theory. In 2011, Kahneman was named b

## Run tool without agent

In [20]:
wikipedia_tool.run("David Beckham")

"Page: David Beckham\nSummary: David Robert Joseph Beckham  ( BEK-əm; born 2 May 1975) is an English former professional footballer, the president and co-owner of Inter Miami CF and co-owner of Salford City. Primarily a right winger and known for his range of passing, crossing ability and free-kick taking, Beckham has been hailed as one of the greatest and most recognisable midfielders of his generation, as well as one of the best free-kick takers of all time. He won 19 major trophies in his career, and is the first English player to win league titles in four different countries: England, Spain, the United States and France.\nBeckham's professional club career began with Manchester United, where he made his first-team debut in 1992 at age 17. With United, he won the Premier League title six times, the FA Cup twice, the FA Charity Shield twice, the Intercontinental Cup and the UEFA Champions League in 1999. He then played four seasons with Real Madrid, winning the La Liga championship i

## Openweathermap Tool

In [4]:
input = {"input": "How is the weather in Salvador da Bahia?"}
tools = [get_openweathermap_tool()]
invoke_cohere_with_tools(input, tools)



> Entering new AgentExecutor chain...

I will search for the weather in Salvador da Bahia and then write an answer.
{'tool_name': 'open_weather_map', 'parameters': {'location': 'Salvador da Bahia'}}
In Salvador da Bahia, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 6.3 m/s, direction: 126°
Humidity: 78%
Temperature: 
  - Current: 23.98°C
  - High: 23.98°C
  - Low: 23.98°C
  - Feels like: 24.47°C
Rain: {}
Heat index: None
Cloud cover: 100%Relevant Documents: 0
Cited Documents: 0
Answer: In Salvador da Bahia, it is currently 23.98°C and overcast. The wind speed is 6.3 m/s and the humidity is 78%.
Grounded answer: In Salvador da Bahia, it is currently <co: 0>23.98°C</co: 0> and <co: 0>overcast</co: 0>. The <co: 0>wind speed is 6.3 m/s</co: 0> and the <co: 0>humidity is 78</co: 0>%.

> Finished chain.


## Multiple Tool Use
1. wiki-tool
2. open_weather_map
3. internet_search

In [5]:
tools = [get_openweathermap_tool(), get_internet_search_tool(), get_wikipedia_tool()]
prompt_template = "Use the right tool to answer the following question. If wikipedia_tool delivers no result use internet_search: {input}"

input = {"input": "Who was George Washington?"}
invoke_cohere_with_tools(input, tools, prompt_template)

input = {"input": "What's the weather like in Mortugaba, Bahia, Brazil?"}
invoke_cohere_with_tools(input, tools, prompt_template)

input = {"input": "Whats in the news today?"}
invoke_cohere_with_tools(input, tools, prompt_template)



> Entering new AgentExecutor chain...

I will use Wikipedia to answer this question.
{'tool_name': 'wikipedia-tool', 'parameters': {'query': 'Who was George Washington?'}}
Page: George Washington Carver
Summary: George Washington Carver (c. 1864 – January 5, 1943) was an American agricultural scientist and inventor who promoted alternative crops to cotton and methods to prevent soil depletion. He was one of the most prominent black scientists of the early 20th century.
While a professor at Tuskegee Institute, Carver developed techniques to improve types of soils depleted by repeated plantings of cotton. He wanted poor farmers to grow other crops, such as peanuts and sweet potatoes, as a source of their own food and to improve their quality of life. Under his leadership, the Experiment Station at Tuskegee published over forty practical bulletins for farmers, many written by him, which included recipes; many of the bulletins contained advice for poor farmers, including combating soil d



> Entering new AgentExecutor chain...

I will use the Open Weather Map tool to find the weather in Mortugaba, Bahia, Brazil.
{'tool_name': 'open_weather_map', 'parameters': {'location': 'Mortugaba, Bahia, Brazil'}}
In Mortugaba, Bahia, Brazil, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 2.1 m/s, direction: 130°
Humidity: 97%
Temperature: 
  - Current: 16.09°C
  - High: 16.09°C
  - Low: 16.09°C
  - Feels like: 16.29°C
Rain: {}
Heat index: None
Cloud cover: 98%Relevant Documents: 0
Cited Documents: 0
Answer: In Mortugaba, Bahia, Brazil, the weather is currently overcast with a temperature of 16.09°C. It's 97% humidity and there's a wind speed of 2.1 m/s.
Grounded answer: In Mortugaba, Bahia, Brazil, the weather is currently <co: 0>overcast</co: 0> with a <co: 0>temperature of 16.09°C</co: 0>. It's <co: 0>97% humidity</co: 0> and there's a <co: 0>wind speed of 2.1 m/s</co: 0>.

> Finished chain.




> Entering new AgentExecutor chain...

I will search for 'news 5 June 2024' and write an answer based on the results.
{'tool_name': 'internet_search', 'parameters': {'query': 'news 5 june 2024'}}
[{'url': 'https://www.bloomberg.com/news/articles/2024-06-04/stock-market-today-dow-s-p-live-updates', 'content': 'The S&P 500 edged up to around 5,290, with futures little changed on Wednesday. The MSCI Emerging Market Index fell the most in seven weeks after India stocks had their worst day in more than four ...'}, {'url': 'https://www.cnn.com/2024/06/04/us/5-things-to-know-for-june-4-hunter-biden-india-election-covid-19-heat-dome-middle-east/index.html', 'content': '4. Heat dome. A heat dome is sending temperatures into the triple-digits across the Western US as wildfires burn in the region. Excessive heat warnings are in effect this week for more than 10 ...'}, {'url': 'https://www.unep.org/events/un-day/world-environment-day-2024', 'content': 'When: 5 June 2024 Theme: Land restoration, 

## Yfinance Tools

In [10]:
tools = [get_business_summary_tool(),get_ticker_history_tool(), get_ticker_info_tool(), get_search_symbol_tool()]

### Business Summary Tool

In [11]:
input = {"input": "What is the stock market symbol for Microsoft? Give me a business summary for the company."}
invoke_cohere_with_tools(input, tools)



> Entering new AgentExecutor chain...

I will first search for the stock market symbol for Microsoft, then I will use this information to find a business summary for the company.
{'tool_name': 'av_search_symbols', 'parameters': {'tool_input': 'stock market symbol for Microsoft'}}
{'bestMatches': []}
I have found that the stock market symbol for Microsoft is MSFT. I will now use this information to find a business summary for the company.
{'tool_name': 'yf_get_business_summary', 'parameters': {'ticker_symbol': 'MSFT'}}
Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solut

### Ticker History Tool

In [21]:
input = {"input": "How did BRK-B perform the last 5d?"}
invoke_cohere_with_tools(input, tools)



> Entering new AgentExecutor chain...

I will search for the ticker history of BRK-B for the last 5 days.
{'tool_name': 'yf_get_ticker_history', 'parameters': {'ticker_symbol': 'BRK-B', 'period': '5d'}}
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Open</th>
      <th>High</th>
      <th>Low</th>
      <th>Close</th>
      <th>Volume</th>
      <th>Dividends</th>
      <th>Stock Splits</th>
    </tr>
    <tr>
      <th>Date</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2024-05-29 00:00:00-04:00</th>
      <td>403.000000</td>
      <td>405.869995</td>
      <td>401.579987</td>
      <td>404.089996</td>
      <td>3426800</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>2024-05-30 00:00:00-04:00</th>
      <td>403.769989</td>
      <td>409.040009</td>
      <td>402.769989</td>
      

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-05-29 00:00:00-04:00,403.000000,405.869995,401.579987,404.089996,3426800,0.0,0.0
2024-05-30 00:00:00-04:00,403.769989,409.040009,402.769989,408.609985,2905300,0.0,0.0
2024-05-31 00:00:00-04:00,408.089996,415.399994,406.880005,414.399994,5190000,0.0,0.0
2024-06-03 00:00:00-04:00,414.890015,416.720001,408.779999,414.790009,4903000,0.0,0.0
2024-06-04 00:00:00-04:00,414.890015,415.890015,406.440094,406.869995,1762127,0.0,0.0


### Ticker Info Tool

In [12]:
prompt_template = "I need detailed information about {ticker_symbol}. Produce a key value list for the 25 most important financial informations using CRLF before each point."
tools = [get_ticker_info_tool()]
input = {"ticker_symbol": "META"}

invoke_cohere_with_tools(input, tools, prompt_template)



> Entering new AgentExecutor chain...

I will use the 'yf_get_ticker_info' tool to find the information requested by the user.
{'tool_name': 'yf_get_ticker_info', 'parameters': {'ticker_symbol': 'META'}}
{'address1': '1 Meta Way', 'city': 'Menlo Park', 'state': 'CA', 'zip': '94025', 'country': 'United States', 'phone': '650 543 4800', 'website': 'https://investor.fb.com', 'industry': 'Internet Content & Information', 'industryKey': 'internet-content-information', 'industryDisp': 'Internet Content & Information', 'sector': 'Communication Services', 'sectorKey': 'communication-services', 'sectorDisp': 'Communication Services', 'longBusinessSummary': 'Meta Platforms, Inc. engages in the development of products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality headsets, and wearables worldwide. It operates in two segments, Family of Apps and Reality Labs. The Family of Apps segment offers Facebook, which enables pe

## Alpha Vantage Tool

In [16]:
tools = [
    # get_search_symbol_tool(),
    get_exchange_rate_tool(),
    # get_time_series_daily_tool(),
    # get_time_series_weekly_tool(),
    # get_market_news_sentimen_tool(),
    # get_top_gainers_losers_tool(),
]
# alpha_vantage = AlphaVantageAPIWrapper()
# alpha_vantage._get_exchange_rate("USD", "JPY")
# alpha_vantage._get_time_series_daily("MSFT")
# alpha_vantage._get_time_series_weekly("IBM")
# alpha_vantage.search_symbols("American Superconductor")
# alpha_vantage._get_market_news_sentiment("Microsoft")
# alpha_vantage._get_top_gainers_losers()
# response = agent_executor.invoke(
#     {"input": "What is the symbol for American Superconductor?"}
# )
input = {"input": "How much is one USD in EUR?"}
invoke_cohere_with_tools(input, tools, no_markdown= True)
# response = agent_executor.invoke({"input": "Who are the gainers and losers at the stock exchange?"})
# response = agent_executor.invoke({"input": "How is Microsoft performing historically?"})
# response = agent_executor.invoke({"input": "How did Apple perform the last time?"})
# response = agent_executor.invoke({"input": "What is in the news about Nvidia?"})



> Entering new AgentExecutor chain...

I will use the currency conversion tool to find out how much one USD is in EUR.
{'tool_name': 'av_get_exchange_rate', 'parameters': {'from_currency': 'USD', 'to_currency': 'EUR'}}
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD', '2. From_Currency Name': 'United States Dollar', '3. To_Currency Code': 'EUR', '4. To_Currency Name': 'Euro', '5. Exchange Rate': '0.91900000', '6. Last Refreshed': '2024-06-05 04:32:35', '7. Time Zone': 'UTC', '8. Bid Price': '0.91890000', '9. Ask Price': '0.91900000'}}


> Finished chain.
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD', '2. From_Currency Name': 'United States Dollar', '3. To_Currency Code': 'EUR', '4. To_Currency Name': 'Euro', '5. Exchange Rate': '0.91900000', '6. Last Refreshed': '2024-06-05 04:32:35', '7. Time Zone': 'UTC', '8. Bid Price': '0.91890000', '9. Ask Price': '0.91900000'}}


## Polygon Toolkit

### Polygon Tools

In [3]:
polygon = PolygonAPIWrapper()
toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)
tools = toolkit.get_tools()
for tool in tools:
    schema = str(tool.args_schema.schema())
    tool_documentation = f"<h1>{tool.name}</h1>\n\n<h2>Description</h2>{tool.description}\n\n<h2>Schema</h2>{schema}"
    display(HTML(markdown.markdown(tool_documentation)))

### Aggregates

In [92]:
# Poloygon is called with the correct arguments but this produces an API error
# input = {"input": "How did MSFT perform the last 5 days?"}
# invoke_cohere_with_tools(tools=tools,input=input)

In [93]:
from langchain_community.tools.polygon.aggregates import PolygonAggregates
from langchain_community.tools.polygon.financials import PolygonFinancials
from langchain_community.tools.polygon.last_quote import PolygonLastQuote
from langchain_community.tools.polygon.ticker_news import PolygonTickerNews
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools.polygon.aggregates import PolygonAggregatesSchema

api_wrapper = PolygonAPIWrapper()
ticker = "AAPL"

# Define param
params = PolygonAggregatesSchema(
    ticker=ticker,
    timespan="day",
    timespan_multiplier=1,
    from_date="2024-03-01",
    to_date="2024-03-08",
)
# Get aggregates for ticker
aggregates_tool = PolygonAggregates(api_wrapper=api_wrapper)
aggregates = aggregates_tool.run(tool_input=params.dict())
print(aggregates)

[{"v": 73450582.0, "vw": 179.0322, "o": 179.55, "c": 179.66, "h": 180.53, "l": 177.38, "t": 1709269200000, "n": 911077}, {"v": 81505451.0, "vw": 174.8938, "o": 176.15, "c": 175.1, "h": 176.9, "l": 173.79, "t": 1709528400000, "n": 1167166}, {"v": 94702355.0, "vw": 170.3234, "o": 170.76, "c": 170.12, "h": 172.04, "l": 169.62, "t": 1709614800000, "n": 1108820}, {"v": 68568907.0, "vw": 169.5506, "o": 171.06, "c": 169.12, "h": 171.24, "l": 168.68, "t": 1709701200000, "n": 896297}, {"v": 71763761.0, "vw": 169.3619, "o": 169.15, "c": 169, "h": 170.73, "l": 168.49, "t": 1709787600000, "n": 825405}, {"v": 76267041.0, "vw": 171.5322, "o": 169, "c": 170.73, "h": 173.7, "l": 168.94, "t": 1709874000000, "n": 925213}]


### Last Quote

In [94]:
# Works only with premium subscription at Polygon
# input = {"input": "What is the end of day price for MSFT?"}
# invoke_cohere_with_tools(tools=tools,input=input)

### Ticker News Sentiment

In [5]:
prompt_template = "What is the news sentiment for {ticker}? Rate the sentiment based on the news and explain your rating."
ticker = "AMSC"
input = {"ticker": ticker}
invoke_cohere_with_tools(tools=tools,input=input,prompt_template=prompt_template)



> Entering new AgentExecutor chain...

I will search for the news sentiment for AMSC and rate the sentiment based on the news.
{'tool_name': 'polygon_ticker_news', 'parameters': {'query': 'AMSC'}}
[{"id": "IaogJOBz1n-HOyXl9XlsGiM0pyYByQrDejQF2yEZ-mc", "publisher": {"name": "The Motley Fool", "homepage_url": "https://www.fool.com/", "logo_url": "https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg", "favicon_url": "https://s3.polygon.io/public/assets/news/favicons/themotleyfool.ico"}, "title": "Why American Superconductor Stock Shocked Investors on Thursday", "author": "newsfeedback@fool.com (Rich Smith)", "published_utc": "2024-05-30T15:46:51Z", "article_url": "https://www.fool.com/investing/2024/05/30/why-american-superconductor-stock-shocked-investor/", "tickers": ["AMSC"], "image_url": "https://g.foolcdn.com/editorial/images/779058/1-glowing-green-stock-arrow-climbs-on-a-stock-screen.jpg", "description": "AMSC isn't a profitable company yet, but it's moving in the righ

### Financials

In [6]:
prompt_template = "How are the financials for {ticker}? What is its market value? Produce an overview in form of a bullet point list (use CRLF for new lines) and rate the financial situation of the company."

input = {"ticker": "AMSC"}
invoke_cohere_with_tools(tools=tools,input=input,prompt_template=prompt_template)



> Entering new AgentExecutor chain...

I will search for AMSC's financials and market value and then write an overview of the company's financial situation.
{'tool_name': 'polygon_financials', 'parameters': {'query': 'AMSC'}}
[{"start_date": "2023-04-01", "end_date": "2024-03-31", "filing_date": "2024-05-29", "acceptance_datetime": "2024-05-29T20:35:54Z", "timeframe": "ttm", "fiscal_period": "TTM", "fiscal_year": "2024", "cik": "0000880807", "sic": "3621", "tickers": ["AMSC", "FMD"], "company_name": "AMERICAN SUPERCONDUCTOR CORP /DE/", "source_filing_url": "https://api.polygon.io/v1/reference/sec/filings/0001437749-24-018602", "source_filing_file_url": "http://api.polygon.io/v1/reference/sec/filings/0001437749-24-018602/files/amsc20240331_10k_htm.xml", "financials": {"cash_flow_statement": {"net_cash_flow_from_investing_activities_continuing": {"value": -961000, "unit": "USD", "label": "Net Cash Flow From Investing Activities, Continuing", "order": 500}, "net_cash_flow_from_operating

### Financials, News and Technical Combined

## Imports for Embeddings and Qdrant

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Qdrant
from langchain_community.vectorstores import Qdrant
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
import qdrant_client

## Build Qdrant Vector Store

In [ ]:
# web_links = ["https://www.fielmann-group.com/ueber-uns/?",
#              "https://de.wikipedia.org/wiki/Fielmann",
#              "https://de.wikipedia.org/wiki/G%C3%BCnther_Fielmann",
#              "https://de.wikipedia.org/wiki/Marc_Fielmann"] 

web_links = ["https://python.langchain.com/v0.2/docs/introduction",
             "https://python.langchain.com/v0.2/docs/concepts",
             "https://python.langchain.com/v0.2/docs/integrations/tools",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores",
             "https://python.langchain.com/v0.2/docs/integrations/document_loaders",
             "https://python.langchain.com/v0.2/docs/integrations/retrievers",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores/qdrant",
             "https://python.langchain.com/v0.2/docs/integrations/providers/cohere/#react-agent",
             "https://de.wikipedia.org/wiki/Herz#:~:text=Das%20Herz%20(lateinisch%20Cor%2C%20griechisch,die%20Versorgung%20aller%20Organe%20sichert."]
 

loader = WebBaseLoader(web_links)
document=loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(document)

embeddings = CohereEmbeddings(model = "embed-english-v2.0")

qdrant = Qdrant.from_documents(
    document,
    embeddings,
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
    collection_name="rag_documents",
    force_recreate=True,
)


## Load Qdrant Vector Store

In [37]:
client = qdrant_client.QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
)
embeddings = CohereEmbeddings(model="embed-english-v2.0")
vector_store = Qdrant(
    client = client,
    collection_name = "rag_documents",
    embeddings = embeddings
)
print("connection established !")

connection established !


## Use Qdrant Vector Store as RAG Retriever for Cohere

In [38]:
prompt_template = """
You are a bot to answer questions from a document.
    
You will be given a context of the conversation made so far followed by a question, 
give the answer to the question using the context. 
The answer should be short, straight and to the point. If you don't know the answer, reply that the answer is not available.

Context: {context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = { "prompt" : PROMPT }

llm = ChatCohere()

template = (
        """Combine the chat history and follow up question into 
        a standalone question. 
        If chat hsitory is empty, use the follow up question as it is.
        Chat History: {chat_history}
        Follow up question: {question}"""
    )

prompt = PromptTemplate.from_template(template)

chain = ConversationalRetrievalChain.from_llm(     
    llm = llm,
    chain_type = "stuff",
    memory = ConversationSummaryMemory(llm = llm, memory_key='chat_history', input_key='question', output_key= 'answer', return_messages=True),
    retriever = vector_store.as_retriever(search_type="mmr"),
    condense_question_prompt = prompt,
    return_source_documents=False,
    combine_docs_chain_kwargs=chain_type_kwargs,
)
customer_prompt = "What are tools in LangChain?"


llm_response = chain.invoke(
                {
                    "context": chain.memory.buffer,
                    "question": customer_prompt,
                },
                return_only_outputs=True,
            )

display(HTML(markdown.markdown(llm_response['answer'])))
